In [1]:
# Importacion de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import random
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


%matplotlib inline

plt.style.use('default') # para graficos matplotlib
plt.rcParams['figure.figsize'] = (5, 5)

sns.set(style="whitegrid") # grid seaborn

# pd.options.display.float_format = '{:20,.0f}'.format # notacion output

# Acceso a set de datos

In [2]:
path = "data/"
df_props_full = pd.read_csv(path + "train.csv")
df_test = pd.read_csv(path + "test.csv")

# Conversión de tipos

In [3]:
df_props_full['fecha'] = pd.to_datetime(df_props_full['fecha'])
df_test['fecha'] = pd.to_datetime(df_test['fecha'])

In [ ]:
# Convierto todos los valores 1/0 a uint8
df_props_full['gimnasio'] = df_props_full['gimnasio'].astype('uint8')
df_props_full['usosmultiples'] = df_props_full['usosmultiples'].astype('uint8')
df_props_full['piscina'] = df_props_full['piscina'].astype('uint8')
df_props_full['escuelascercanas'] = df_props_full['escuelascercanas'].astype('uint8')
df_props_full['centroscomercialescercanos'] = df_props_full['centroscomercialescercanos'].astype('uint8')

df_test['gimnasio'] = df_test['gimnasio'].astype('uint8')
df_test['usosmultiples'] = df_test['usosmultiples'].astype('uint8')
df_test['piscina'] = df_test['piscina'].astype('uint8')
df_test['escuelascercanas'] = df_test['escuelascercanas'].astype('uint8')
df_test['centroscomercialescercanos'] = df_test['centroscomercialescercanos'].astype('uint8')

In [ ]:
# Convierto los representables en uint8. Utilizo el tipo de pandas UInt8Dtype para evitar conflicto con NaN
df_props_full['antiguedad'] = df_props_full['antiguedad'].astype(pd.UInt8Dtype())
df_props_full['habitaciones'] = df_props_full['habitaciones'].astype(pd.UInt8Dtype())
df_props_full['garages'] = df_props_full['garages'].astype(pd.UInt8Dtype())
df_props_full['banos'] = df_props_full['banos'].astype(pd.UInt8Dtype())

df_test['antiguedad'] = df_test['antiguedad'].astype(pd.UInt8Dtype())
df_test['habitaciones'] = df_test['habitaciones'].astype(pd.UInt8Dtype())
df_test['garages'] = df_test['garages'].astype(pd.UInt8Dtype())
df_test['banos'] = df_test['banos'].astype(pd.UInt8Dtype())

In [ ]:
# Convierto los representables en uint16. Utilizo el tipo de pandas UInt16Dtype para evitar conflicto con NaN
df_props_full['metroscubiertos'] = df_props_full['metroscubiertos'].astype(pd.UInt16Dtype())
df_props_full['metrostotales'] = df_props_full['metrostotales'].astype(pd.UInt16Dtype())

df_test['metroscubiertos'] = df_test['metroscubiertos'].astype(pd.UInt16Dtype())
df_test['metrostotales'] = df_test['metrostotales'].astype(pd.UInt16Dtype())

In [ ]:
# Convierto los representables en uint32. Utilizo el tipo de pandas UInt32Dtype para evitar conflicto con NaN
df_props_full['id'] = df_props_full['id'].astype(pd.UInt32Dtype())
df_props_full['idzona'] = df_props_full['idzona'].astype(pd.UInt32Dtype())
# df_props_full['Precio_MEX'] = df_props_full['Precio_MEX'].astype(pd.UInt32Dtype())
# df_props_full['Precio_USD'] = df_props_full['Precio_USD'].astype(pd.UInt32Dtype())
df_props_full['precio'] = df_props_full['precio'].astype('float32')

df_test['id'] = df_test['id'].astype(pd.UInt32Dtype())
df_test['idzona'] = df_test['idzona'].astype(pd.UInt32Dtype())

In [ ]:
features = df_props_full
len(features)

In [ ]:
features.corr()

# Dropna Tipo de propiedad

In [ ]:
features.tipodepropiedad.isna().value_counts()

In [ ]:
# Drop de datos sin tipo
features = features.dropna(subset=['tipodepropiedad'])
features = features.reset_index(drop=True)

len(features)

# Metros

In [ ]:
print('Metrocubiertos con datos nulos: ', features.metroscubiertos.isna().sum())
print('Metrocubiertos con datos nulos: ', features.metrostotales.isna().sum())
print('Publicaciones sin metros cubiertos y totales: ', len(features.loc[(features.metroscubiertos.isna() & features.metrostotales.isna()),:]))
print('Publicaciones con metroscubiertos==metrostotales: ', len(features.loc[features.metroscubiertos == features.metrostotales, :]))
print('Publicaciones con metroscubiertos>metrostotales (Error): ', len(features.loc[features.metroscubiertos > features.metrostotales, ['metroscubiertos','metrostotales']]))

Swapeo columnas metroscubiertos-metrostotales en el caso que la primera es mayor a la segunda

In [ ]:
cond = features.metroscubiertos > features.metrostotales
features.loc[cond, ['metroscubiertos', 'metrostotales']] = features.loc[cond, ['metrostotales', 'metroscubiertos']].values

In [ ]:
print('Publicaciones con metroscubiertos>metrostotales (Error): ', len(features.loc[features.metroscubiertos > features.metrostotales, ['metroscubiertos','metrostotales']]))

In [ ]:
features['tot-cub'] = features.metrostotales - features.metroscubiertos

In [ ]:
# Calculo diferencia promedio entre metrostotales y metroscubiertos por cada tipo de propiedad
meanDifByType = features.groupby('tipodepropiedad').agg({'tot-cub':'mean'}).fillna(0).T.round(0).astype(np.uint16)
meanDifByType

In [ ]:
# Calculo diferencia media entre metrostotales y metroscubiertos por cada tipo de propiedad
medianDifByType = features.groupby('tipodepropiedad').agg({'tot-cub':'median'}).fillna(0).T.round(0).astype(np.uint16)
medianDifByType

In [ ]:
def getMetrosCubiertos(row):
    if (np.isnan(row.metroscubiertos)):
        return (row.metrostotales - (medianDifByType.loc['tot-cub',row.tipodepropiedad])).astype(np.uint16)
    return row.metroscubiertos

In [ ]:
def getMetrosTotales(row):
    if (np.isnan(row.metrostotales)):
        return (row.metroscubiertos + medianDifByType.loc['tot-cub',row.tipodepropiedad]).astype(np.uint16)
    return row.metrostotales

In [ ]:
features['metroscubiertos'] = features.apply(lambda row: getMetrosCubiertos(row), axis=1)
features['metrostotales'] = features.apply(lambda row: getMetrosTotales(row), axis=1)

In [ ]:
print('Metrocubiertos con datos nulos: ', features.metroscubiertos.isna().sum())
print('Metrostotales con datos nulos: ', features.metrostotales.isna().sum())

### Set Test

In [ ]:
print('Metrocubiertos con datos nulos: ', df_test.metroscubiertos.isna().sum())
print('Metrocubiertos con datos nulos: ', df_test.metrostotales.isna().sum())
print('Publicaciones sin metros cubiertos y totales: ', len(df_test.loc[(df_test.metroscubiertos.isna() & df_test.metrostotales.isna()),:]))
print('Publicaciones con metroscubiertos==metrostotales: ', len(df_test.loc[df_test.metroscubiertos == df_test.metrostotales, :]))
print('Publicaciones con metroscubiertos>metrostotales (Error): ', len(df_test.loc[df_test.metroscubiertos > df_test.metrostotales, ['metroscubiertos','metrostotales']]))

In [ ]:
print('Publicaciones Test con tipopropiedad nula: ', df_test.tipodepropiedad.isna().sum())

In [ ]:
def getMetrosCubiertosTest(row):
    if (np.isnan(row.metroscubiertos)):
        if (not(pd.isnull(row.tipodepropiedad))):
            return (row.metrostotales - (medianDifByTypeTest.loc['tot-cub',row.tipodepropiedad])).astype(np.uint16)
        else:
            return row.metrostotales
    return row.metroscubiertos

In [ ]:
def getMetrosTotalesTest(row):
    if (np.isnan(row.metrostotales)):
        if (not(pd.isnull(row.tipodepropiedad))):
            return (row.metroscubiertos + medianDifByTypeTest.loc['tot-cub',row.tipodepropiedad]).astype(np.uint16)
        else:
            return row.metroscubiertos
    return row.metrostotales

In [ ]:
cond = df_test.metroscubiertos > df_test.metrostotales
df_test.loc[cond, ['metroscubiertos', 'metrostotales']] = df_test.loc[cond, ['metrostotales', 'metroscubiertos']].values
df_test['tot-cub'] = df_test.metrostotales - df_test.metroscubiertos

medianDifByTypeTest = df_test.groupby('tipodepropiedad').agg({'tot-cub':'median'}).fillna(0).T.round(0).astype(np.uint16)
medianDifByTypeTest

df_test['metroscubiertos'] = df_test.apply(lambda row: getMetrosCubiertosTest(row), axis=1)
df_test['metrostotales'] = df_test.apply(lambda row: getMetrosTotalesTest(row), axis=1)

# for index, row in df_test.iterrows():
#     if (np.isnan(row.metroscubiertos)):
#         if (not(pd.isnull(row.tipodepropiedad))):
#             df_test.loc[index,'metroscubiertos'] = (row.metrostotales - (medianDifByTypeTest.loc['tot-cub',row.tipodepropiedad])).astype(np.uint16)
#         else:
#             df_test.loc[index,'metroscubiertos'] = row.metrostotales
#     if (np.isnan(row.metrostotales)):
#         if (not(pd.isnull(row.tipodepropiedad))):
#             df_test.loc[index,'metrostotales'] = (row.metroscubiertos + medianDifByTypeTest.loc['tot-cub',row.tipodepropiedad]).astype(np.uint16)
#         else:
#             df_test.loc[index,'metrostotales'] = row.metroscubiertos
        
print('Metrocubiertos con datos nulos: ', df_test.metroscubiertos.isna().sum())
print('Metrostotales con datos nulos: ', df_test.metrostotales.isna().sum())

# Fecha

In [ ]:
features['anio'] = features.fecha.dt.year
features['mes'] = features.fecha.dt.month

df_test['anio'] = df_test.fecha.dt.year
df_test['mes'] = df_test.fecha.dt.month

# Habitaciones

In [ ]:
print('Habitaciones con datos nulos: ', features.habitaciones.isna().sum())

In [ ]:
pd.pivot_table(features.groupby(['tipodepropiedad','habitaciones']).size().to_frame(), index='tipodepropiedad',columns=['habitaciones'])

In [ ]:
print('Cantidad valores nulos en cada propiedad')
features.groupby(['tipodepropiedad']).agg({'habitaciones':lambda x: x.isna().sum()}).sort_values(by='habitaciones', ascending=True).plot(kind='barh')

In [ ]:
print('Porcentaje valores no nulos en cada propiedad')
features.groupby(['tipodepropiedad']).agg({'habitaciones':lambda x: x.count() / len(x) * 100}).sort_values(by='habitaciones', ascending=True).plot(kind='barh')

# Habitaciones, Baños, Garages y Antiguedad

In [ ]:
print('Cantidad publicaciones Lote: ', len(features.loc[features.tipodepropiedad=='Lote','habitaciones']))
print('Datos Habitaciones: ', features.loc[features.tipodepropiedad=='Lote','habitaciones'].count())
print('Datos Baños: ', features.loc[features.tipodepropiedad=='Lote','banos'].count())
print('Datos Garages: ', features.loc[features.tipodepropiedad=='Lote','garages'].count())

In [ ]:
print('Cantidad publicaciones Garage: ', len(features.loc[features.tipodepropiedad=='Garage','habitaciones']))
print('Datos Habitaciones: ', features.loc[features.tipodepropiedad=='Garage','habitaciones'].count())
print('Datos Baños: ', features.loc[features.tipodepropiedad=='Garage','banos'].count())
print('Datos Garages: ', features.loc[features.tipodepropiedad=='Garage','garages'].count())

In [ ]:
print('Cantidad publicaciones Hospedaje: ', len(features.loc[features.tipodepropiedad=='Hospedaje','habitaciones']))
print('Datos Habitaciones: ', features.loc[features.tipodepropiedad=='Hospedaje','habitaciones'].count())
print('Datos Baños: ', features.loc[features.tipodepropiedad=='Hospedaje','banos'].count())
print('Datos Garages: ', features.loc[features.tipodepropiedad=='Hospedaje','garages'].count())

In [ ]:
features = features.loc[(features.tipodepropiedad!='Lote') & (features.tipodepropiedad!='Garage') & (features.tipodepropiedad!='Hospedaje'),:]

In [ ]:
print('Cantidad publicaciones Lote: ', len(features.loc[features.tipodepropiedad=='Lote','habitaciones']))
print('Cantidad publicaciones Garage: ', len(features.loc[features.tipodepropiedad=='Garage','habitaciones']))

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [ ]:
import warnings
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [ ]:
for tipo in features.tipodepropiedad.unique():
    features.loc[features.tipodepropiedad==tipo,['habitaciones','banos','garages','antiguedad']] = imputer.fit_transform(features.loc[features.tipodepropiedad==tipo,['habitaciones','banos','garages','antiguedad']])

### Set Test

In [ ]:
df_test.tipodepropiedad.isna().sum()

In [ ]:
df_test.loc[df_test.tipodepropiedad.isna(),['habitaciones','banos','garages']]

In [ ]:
# Seteo manualmente casa, en base a informacion que se tiene (o no) de los datos de tipo nulo
df_test.loc[df_test.tipodepropiedad.isna(),'tipodepropiedad'] = 'Casa'

In [ ]:
print('Cantidad publicaciones Terreno industrial: ', len(df_test.loc[df_test.tipodepropiedad=='Terreno industrial','habitaciones']))
print('Datos Habitaciones: ', df_test.loc[df_test.tipodepropiedad=='Terreno industrial','habitaciones'].count())
print('Datos Baños: ', df_test.loc[df_test.tipodepropiedad=='Terreno industrial','banos'].count())
print('Datos Garages: ', df_test.loc[df_test.tipodepropiedad=='Terreno industrial','garages'].count())

In [ ]:
# En base a informacion que se tiene de Terreno y Terreno comercial completo
df_test.loc[df_test.tipodepropiedad=='Terreno industrial','habitaciones'] = 3
df_test.loc[df_test.tipodepropiedad=='Terreno industrial','banos'] = 1
df_test.loc[df_test.tipodepropiedad=='Terreno industrial','garages'] = 0

In [ ]:
print('Cantidad publicaciones Lote: ', len(df_test.loc[df_test.tipodepropiedad=='Lote','habitaciones']))
print('Datos Habitaciones: ', df_test.loc[df_test.tipodepropiedad=='Lote','habitaciones'].count())
print('Datos Baños: ', df_test.loc[df_test.tipodepropiedad=='Lote','banos'].count())
print('Datos Garages: ', df_test.loc[df_test.tipodepropiedad=='Lote','garages'].count())

In [ ]:
# Completo con 0 dado que no tengo informacion y las caracteristicas usuales de un lote
df_test.loc[df_test.tipodepropiedad=='Lote','habitaciones'] = 0
df_test.loc[df_test.tipodepropiedad=='Lote','banos'] = 0
df_test.loc[df_test.tipodepropiedad=='Lote','garages'] = 0

In [ ]:
for tipo in df_test.tipodepropiedad.unique():
    df_test.loc[df_test.tipodepropiedad==tipo,['habitaciones','banos','garages','antiguedad']] = imputer.fit_transform(df_test.loc[df_test.tipodepropiedad==tipo,['habitaciones','banos','garages','antiguedad']])

# Provincia

In [ ]:
Noroeste = ['Baja California Norte', 'Baja California Sur','Chihuahua', 'Durango', 'Sinaloa', 'Sonora']
Noreste = ['Coahuila','Nuevo León', 'Tamaulipas']
Oeste = ['Colima', 'Jalisco', 'Michoacán', 'Nayarit']
Este = ['Hidalgo', 'Puebla', 'Tlaxcala', 'Veracruz']
Centronorte = ['Aguascalientes', 'Guanajuato', 'Querétaro', 'San luis Potosí', 'Zacatecas']
Centrosur = ['Edo. de México', 'Distrito Federal', 'Morelos']
Suroeste = ['Chiapas', 'Guerrero', 'Oaxaca']
Sureste = ['Campeche', 'Quintana Roo', 'Tabasco', 'Yucatán']

In [ ]:
def region(provincia):
    if(provincia in Noroeste):
        return 'Noroeste'
    if(provincia in Noreste):
        return 'Noreste'
    if(provincia in Oeste):
        return 'Oeste'
    if(provincia in Este):
        return 'Este'
    if(provincia in Centronorte):
        return 'Centronorte'
    if(provincia in Centrosur):
        return 'Centrosur'
    if(provincia in Suroeste):
        return 'Suroeste'
    if (provincia in Sureste):
        return 'Sureste'
    # provincia nula
    return random.choice(['Noroeste','Noreste','Oeste','Este','Centronorte','Centrosur','Suroeste','Sureste'])

In [ ]:
print('Publicaciones sin provincia: ', features.provincia.isna().sum())
print('Publicaciones sin provincia ni ciudad ni idzona: ', len(features.loc[(features.provincia.isna() & features.ciudad.isna() & features.idzona.isna()),:]))

In [ ]:
# No tengo ningun dato para obtener provincia y cantidad es muy baja
features = features.dropna(subset=['provincia'])
features = features.reset_index(drop=True)

In [ ]:
# One-hot regiones
features['region'] = features['provincia'].apply(region)
one_hot = pd.get_dummies(features.region)
features = features.merge(one_hot, right_index=True, left_index=True)

### Set Test

In [ ]:
print('Publicaciones sin provincia: ', df_test.provincia.isna().sum())
print('Publicaciones sin provincia ni ciudad ni idzona: ', len(df_test.loc[(df_test.provincia.isna() & df_test.ciudad.isna() & df_test.idzona.isna()),:]))

In [ ]:
df_test['region'] = df_test['provincia'].apply(region)
print('Publicaciones sin region: ', df_test.region.isna().sum())

In [ ]:
one_hot_test = pd.get_dummies(df_test.region)
df_test = df_test.merge(one_hot_test, right_index=True, left_index=True)

#### Pruebas previas

# Tipo de propiedad

In [ ]:
features_tipos = features.tipodepropiedad.value_counts().index
print(len(features_tipos))
features_tipos.sort_values()

In [ ]:
one_hot = pd.get_dummies(features.tipodepropiedad)
features = features.merge(one_hot, right_index=True, left_index=True)

In [ ]:
test_tipos = df_test.tipodepropiedad.value_counts().index
print(len(test_tipos))
test_tipos.sort_values()

In [ ]:
one_hot_test = pd.get_dummies(df_test.tipodepropiedad)
df_test = df_test.merge(one_hot_test, right_index=True, left_index=True)

In [ ]:
print(list(set(features.columns) - set(df_test.columns)))
print(list(set(df_test.columns) - set(features.columns)))

In [ ]:
df_test.Lote.value_counts()

In [ ]:
# Elimino columnas que no aparecen en los features (Solo una publicacion con Lote)
df_test = df_test.drop('Lote',axis=1)

# Dolar

In [ ]:
df_dollar = pd.read_csv(path + 'dollar.csv')
df_dollar = df_dollar.dropna()
df_dollar['Cierre'] = pd.to_numeric(df_dollar['Cierre'])
df_dollar['Cierre'] = df_dollar['Cierre'].round(3)
df_dollar['Fecha'] = pd.to_datetime(df_dollar['Fecha'], format='%d.%m.%Y')
df_dollar = df_dollar.set_index('Fecha')
df_dollar = df_dollar.loc[:, 'Cierre'].to_frame()

In [ ]:
# Agrego fechas faltantes (Sabados y Domingos) con valor 0
idx = pd.date_range(start='2011-12-12', end='2017-01-31')
df_dollar = df_dollar.reindex(idx, fill_value=0)

In [ ]:
# Cuando se trata de una fecha que corresponde a un Sabado o Domingo no se tiene infromación sobre Cierre
# Le asigno el valor correspondiente al Viernes previo
for i in range(0, len(df_dollar)):
    if (df_dollar.iloc[i]['Cierre'] == 0):
        df_dollar.iloc[i]['Cierre'] = df_dollar.iloc[i-1]['Cierre']

In [ ]:
features['fecha'] = features['fecha'].apply(lambda x: x.replace(hour=0, minute=0, second=0)) # Seteo tiempo a 00:00:00 para join
features = features.set_index('fecha')
features = features.join(df_dollar, how='left')
features = features.reset_index()
features.rename(columns = {'index' : 'fecha'}, inplace=True)

In [ ]:
features.rename(columns = {'Cierre' : 'Valor dolar'}, inplace=True)

### Set test

In [ ]:
df_test['fecha'] = df_test['fecha'].apply(lambda x: x.replace(hour=0, minute=0, second=0)) # Seteo tiempo a 00:00:00 para join
df_test = df_test.set_index('fecha')
df_test = df_test.join(df_dollar, how='left')
df_test = df_test.reset_index()
df_test.rename(columns = {'index' : 'fecha'}, inplace=True)
df_test.rename(columns = {'Cierre' : 'Valor dolar'}, inplace=True)

# Valor metro cuadrado (Tipo)

In [ ]:
print('Publicaciones sin tipo: ', features.tipodepropiedad.isna().sum())

In [ ]:
features['valor m2 tipodepropiedad'] = (features.precio * features['Valor dolar']) / features.metrostotales
valueXType = features.groupby('tipodepropiedad').agg({'valor m2 tipodepropiedad':'mean'})

In [ ]:
def getM2Tipo(row):
    if (not pd.isnull(row.tipodepropiedad) and row.tipodepropiedad in valueXType.index):
        return valueXType.loc[row.tipodepropiedad,'valor m2 tipodepropiedad']
    return valueXType['valor m2 tipodepropiedad'].median()

In [ ]:
features['valor m2 tipodepropiedad'] = features.apply(lambda row: getM2Tipo(row), axis=1)

### Set Test

In [ ]:
print('Publicaciones sin tipodepropiedad: ', df_test.tipodepropiedad.isna().sum())

In [ ]:
df_test['valor m2 tipodepropiedad'] = df_test.apply(lambda row: getM2Tipo(row), axis=1)

In [ ]:
df_test['valor m2 tipodepropiedad'].count()

# Valor metro cuadrado (Ciudad)

In [ ]:
print('Publicaciones sin ciudad: ', features.ciudad.isna().sum())
print('Publicaciones sin idzona: ', features.idzona.isna().sum())
print('Publicaciones sin ciudad ni idzona: ', len(features.loc[features.ciudad.isna() & features.idzona.isna(),:]))
print('Cantidad ciudades: ', len(features.ciudad.unique()))
print('Cantidad idzzonas: ', len(features.idzona.unique()))

Descarto aquellas publicaciones sin ciudad (tan solo 201) ya que tampoco voy a poder obtener valor de idzona

In [ ]:
features = features.dropna(subset=['ciudad'])
print('Publicaciones sin ciudad: ', features.ciudad.isna().sum())
print('Publicaciones sin idzona: ', features.idzona.isna().sum())

In [ ]:
print('Media de idzona por ciduad: ', features.groupby('ciudad').agg({'idzona':lambda x: len(x.unique())}).idzona.median())
print('Promedio de idzona por ciduad: ', features.groupby('ciudad').agg({'idzona':lambda x: len(x.unique())}).idzona.mean())
print('Maximo de idzona por ciudad: ', features.groupby('ciudad').agg({'idzona':lambda x: len(x.unique())}).idzona.max())
print('Minimo de idzona por ciudad: ', features.groupby('ciudad').agg({'idzona':lambda x: len(x.unique())}).idzona.min())

In [ ]:
# DF con cantidad de veces que se repite la cantiad de idzonas por ciudad
print(features.groupby('ciudad').agg({'idzona':lambda x: len(x.unique())}).groupby('idzona').size().to_frame().sum())
features.groupby('ciudad').agg({'idzona':lambda x: len(x.unique())}).groupby('idzona').size().to_frame().T

Dado que es muy baja la cantidad de ciudades con un solo idzona y hay gran cantidad de datos nulos de idzona se decide calcular el valor del metro cuadrado por ciudad (EN DOLARES)

In [ ]:
features = features.reset_index(drop=True)

In [ ]:
features['valor m2 ciudad'] = (features.precio * features['Valor dolar']) / features.metrostotales
valueXCity = features.groupby('ciudad').agg({'valor m2 ciudad':'mean'})

In [ ]:
def getM2(row):
    if (not pd.isnull(row.ciudad) and row.ciudad in valueXCity.index):
        return valueXCity.loc[row.ciudad,'valor m2 ciudad']
    return valueXCity['valor m2 ciudad'].median()

In [ ]:
features['valor m2 ciudad'] = features.apply(lambda row: getM2(row), axis=1)

### Set Test

In [ ]:
print('Publicaciones sin provincia: ', df_test.provincia.isna().sum())
print('Publicaciones sin ciudad: ', df_test.ciudad.isna().sum())
print('Publicaciones sin idzona: ', df_test.idzona.isna().sum())
print('Publicaciones sin ciudad ni idzona: ', len(df_test.loc[df_test.ciudad.isna() & df_test.idzona.isna(),:]))
print('Publicaciones sin ciudad ni provincia: ', len(df_test.loc[df_test.ciudad.isna() & df_test.provincia.isna(),:]))
print('Publicaciones sin ciudad ni idzona ni provincia: ', len(df_test.loc[df_test.ciudad.isna() & df_test.idzona.isna() & df_test.provincia.isna(),:]))
print('Cantidad ciudades: ', len(df_test.ciudad.unique()))
print('Cantidad idzonas: ', len(df_test.idzona.unique()))

In [ ]:
df_test['valor m2 ciudad'] = df_test.apply(lambda row: getM2(row), axis=1)

In [ ]:
df_test['valor m2 ciudad'].count()

# Terminos de Publicaciones mas caras

In [ ]:
print('Precio maximo: ', features.precio.max())
print('Precio minimo: ', features.precio.min())
print('Precio median: ', features.precio.median())
print('Precio prmdio: ', features.precio.mean())
print('Publicaciones con precio mayor a promedio: ', len(features.loc[features.precio > features.precio.mean(),:]))
print('Publicaciones con precio menor o igual a promedio: ', len(features.loc[features.precio <= features.precio.mean(),:]))

# Features a CSV

In [ ]:
cols_dropped = ['lat', 'lng', 'id', 'idzona', 'tot-cub','region','provincia', 'tipodepropiedad']
cols_string = ['titulo', 'descripcion','direccion', 'ciudad','fecha']
features_to_export = features.drop(cols_dropped + cols_string, axis=1)

In [ ]:
cols_dropped = ['lat', 'lng', 'idzona', 'tot-cub','region','provincia', 'tipodepropiedad']
test_to_export = df_test.drop(cols_dropped + cols_string, axis=1)

In [ ]:
test_to_export.count()

In [ ]:
features_to_export.to_csv('data/features.csv', index=False)
test_to_export.to_csv('data/testModified.csv', index=False)